# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [39]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    (str, int, int, int, int, int, float, str, str, str) -> float

    This function calculate the PRIEST score and return the 30-day probability of an outcome.

    >>> priest("Male", 82, 25, 91, 131, 99, 38.3, "Alert", "Air", "Unrestricted Normal Activity")
    0.59

    >>> priest("Female", 83, 28, 88, 140, 223, 40.0, "Confused or Not Alert", "Supplemental Oxygen", "Bed/chair bound, no self-care")
    0.99

    >>> priest("Unknown", 70, 8, 92, 90, 120, 36.5, "Alert", "Air", "Unrestricted Normal Activity") is None
    True
    """

    sex = sex.lower()
    alertness = alertness.lower()
    inspired_oxygen = inspired_oxygen.lower()
    performance_status = performance_status.lower()

    sex_score = 0 if sex == "female" else 1 if sex == "male" else None

    if 16 <= age <= 49:
        age_score = 0
    elif 50 <= age <= 65:
        age_score = 2
    elif 66 <= age <= 80:
        age_score = 3
    elif age > 80:
        age_score = 4
    else:
        age_score = None

    if respiratory_rate < 9:
        resp_rate_score = 3
    elif 9 <= respiratory_rate <= 11:
        resp_rate_score = 1
    elif 12 <= respiratory_rate <= 20:
        resp_rate_score = 0
    elif 21 <= respiratory_rate <= 24:
        resp_rate_score = 2
    elif respiratory_rate > 24:
        resp_rate_score = 3
    else:
        resp_rate_score = None

    if oxygen_saturation > 95:
        oxygen_score = 0
    elif 94 <= oxygen_saturation <= 95:
        oxygen_score = 1
    elif 92 <= oxygen_saturation <= 93:
        oxygen_score = 2
    elif oxygen_saturation < 92:
        oxygen_score = 3
    else:
        oxygen_score = None

    if heart_rate < 41:
        heart_rate_score = 3
    elif 41 <= heart_rate <= 50:
        heart_rate_score = 1
    elif 51 <= heart_rate <= 90:
        heart_rate_score = 0
    elif 91 <= heart_rate <= 110:
        heart_rate_score = 1
    elif 111 <= heart_rate <= 130:
        heart_rate_score = 2
    elif heart_rate > 130:
        heart_rate_score = 3
    else:
        heart_rate_score = None

    if systolic_bp < 91:
        bp_score = 3
    elif 91 <= systolic_bp <= 100:
        bp_score = 2
    elif 101 <= systolic_bp <= 110:
        bp_score = 1
    elif 111 <= systolic_bp <= 219:
        bp_score = 0
    elif systolic_bp > 219:
        bp_score = 3
    else:
        bp_score = None

    if temperature < 35.1:
        temp_score = 3
    elif 35.1 <= temperature <= 36.0:
        temp_score = 1
    elif 36.1 <= temperature <= 38.0:
        temp_score = 0
    elif 38.1 <= temperature <= 39.0:
        temp_score = 1
    elif temperature > 39.0:
        temp_score = 2
    else:
        temp_score = None

    alertness_score = 0 if alertness == "alert" else 3 if alertness == "confused or not alert" else None

    inspired_oxygen_score = 0 if inspired_oxygen == "air" else 2 if inspired_oxygen == "supplemental oxygen" else None

    if performance_status == "unrestricted normal activity":
        performance_status_score = 0
    elif performance_status == "limited strenuous activity, can do light activity":
        performance_status_score = 1
    elif performance_status == "limited activity, can self-care":
        performance_status_score = 2
    elif performance_status == "limited self-care":
        performance_status_score = 3
    elif performance_status == "bed/chair bound, no self-care":
        performance_status_score = 4
    else:
        performance_status_score = None

    # Checking for invalid input values
    if None in [sex_score, age_score, resp_rate_score, oxygen_score, heart_rate_score, bp_score, temp_score, alertness_score, inspired_oxygen_score, performance_status_score]:
        return None

    # Sum up PRIEST score
    priest_score = (
        sex_score
        + age_score
        + resp_rate_score
        + oxygen_score
        + heart_rate_score
        + bp_score
        + temp_score
        + alertness_score
        + inspired_oxygen_score
        + performance_status_score
    )

    # Finding the 30-day probability based on PRIEST score
    if priest_score >= 0 and priest_score <= 1:
        return 0.01
    elif priest_score >= 2 and priest_score <= 3:
        return 0.02
    elif 17 <= priest_score <= 25:
        return 0.59
    elif priest_score >= 26:
        return 0.99
    elif priest_score >= 4:
        return 0.03
    elif priest_score == 5:
        return 0.09
    elif priest_score == 6:
        return 0.15
    elif priest_score == 7:
        return 0.18
    elif priest_score == 8:
        return 0.22
    elif priest_score == 9:
        return 0.26
    elif priest_score == 10:
        return 0.29
    elif priest_score == 11:
        return 0.34
    elif priest_score == 12:
        return 0.38
    elif priest_score == 13:
        return 0.46
    elif priest_score == 14:
        return 0.47
    elif priest_score == 15:
        return 0.49
    elif priest_score == 16:
        return 0.55
    else:
        return None

In [40]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("Male", 82, 25, 91, 131, 99, 38.3, "Alert", "Air", "Unrestricted Normal Activity")
Expecting:
    0.59
ok
Trying:
    priest("Female", 83, 28, 88, 140, 223, 40.0, "Confused or Not Alert", "Supplemental Oxygen", "Bed/chair bound, no self-care")
Expecting:
    0.99
ok
Trying:
    priest("Unknown", 70, 8, 92, 90, 120, 36.5, "Alert", "Air", "Unrestricted Normal Activity") is None
Expecting:
    True
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [41]:
import requests

def find_hospital(age, sex, risk):
    """(int, str, float) -> str
    This function calls a REST web service to determine the hospital for a patient based on age, sex, and risk.

    >>> find_hospital(43, 'male', 0.4)
    'Emory Dunwoody Medical Center'

    >>> find_hospital(72, 'female', 0.02)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(40, 'male', 0.3)
    'Southwest Hospital and Medical Center'
    """
    # Base URL for the REST web service
    base_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"

    # Prepare extended URL by appending parameters in query parameters
    url = f"{base_url}?age={age}&sex={sex}&risk_pct={risk}"

    try:
        # HTTP GET request
        response = requests.get(url)

        # Validate if the request was successful by checking the status code
        if response.status_code == 200:
            data = response.json()
            return data.get("hospital")
        else:
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None


In [42]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(43, 'male', 0.4)
Expecting:
    'Emory Dunwoody Medical Center'
ok
Trying:
    find_hospital(72, 'female', 0.02)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(40, 'male', 0.3)
Expecting:
    'Southwest Hospital and Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [43]:
import json

def get_address(hospital_name):
    """(str) -> str
    This function retrieves the address of a hospital in Atlanta region in Georgia based on its name.

    Example:
    >>> get_address("WELLSTAR SPALDING REGIONAL HOSPITAL")
    '601 SOUTH 8TH STREET'

    >>> get_address("LOUIS SMITH MEMORIAL HOSPITAL")
    '116 WEST THIGPEN AVENUE'

    >>> get_address("CHILDRENS HOSPITAL NAVICENT HEALTH")
    '888 PINE STREET'
    """
    # URL for the JSON file containing hospital data
    json_url = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"

    try:
        # Fetch the JSON file
        response = requests.get(json_url)

        if response.status_code == 200:
            hospitals_data = json.loads(response.text)

            # Look for the hospital by its name
            hospital_data = hospitals_data.get(hospital_name)

            # If the hospital is found then return its address
            if hospital_data:
                return hospital_data.get("ADDRESS")

            return None
        else:
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

In [44]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("WELLSTAR SPALDING REGIONAL HOSPITAL")
Expecting:
    '601 SOUTH 8TH STREET'
ok
Trying:
    get_address("LOUIS SMITH MEMORIAL HOSPITAL")
Expecting:
    '116 WEST THIGPEN AVENUE'
ok
Trying:
    get_address("CHILDRENS HOSPITAL NAVICENT HEALTH")
Expecting:
    '888 PINE STREET'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [45]:
def process_people(file_location):
    """(str) -> json
    Process patient data from a pipe-delimited file and return a list of dictionaries.
    """

    response = requests.get(file_location)

    if response.status_code != 200:
        return []

    data = response.text.splitlines()[1:]
    result = []

    for line in data:
        # Split the line into fields
        fields = line.strip().split('|')

        if len(fields) >= 11:
            patient_number, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status = fields[:11]
            risk = priest(sex, int(age), int(breath), int(float(o2sat) * 100), int(heart), int(systolic), float(temp), alertness, inspired, status)
            hospital = find_hospital(age, sex.lower(), risk)
            address = get_address(hospital.upper())
            patient_data = {
                patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]
            }
            result.append(patient_data)

    return result

In [46]:
print(process_people("https://hds5210-data.s3.amazonaws.com/people.psv"))

[{'E9559': ['FEMALE', '40', '24', '0.96', '105', '115', '34.9', 'ALERT', 'AIR', 'unrestricted normal activity', 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']}, {'E9385': ['Female', '51', '19', '0.99', '90', '91', '38.1', 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']}, {'E3067': ['female', '40', '29', '0.96', '105', '95', '38.1', 'ALERT', 'air', 'unrestricted normal activity', 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']}, {'E9422': ['FEMALE', '66', '19', '0.96', '135', '115', '38.1', 'ALERT', 'AIR', 'bed/chair bound, no self-care', 'Wesley Woods Geriatric Hospital', '1821 CLIFTON ROAD, NE']}, {'E8661': ['MALE', '18', '22', '0.95', '105', '115', '36.3', 'Confused or Not Alert', 'air', 'unrestricted normal activity', 'Southwest Hospital and Medical Center', '501 FAIRBURN ROAD SW']}, {'E6235': ['Male', '16', '23', '0.96', '135', '115', '39.2', '

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [47]:
# Load data
file_location = "https://hds5210-data.s3.amazonaws.com/people.psv"
your_results = process_people(file_location)

# Load expected results from people_results.json
expected_results_url = "https://hds5210-data.s3.amazonaws.com/people_results.json"
response = requests.get(expected_results_url)

if response.status_code == 200:
    expected_results = json.loads(response.text)

    correct_results = 0
    incorrect_results = 0

    for data_dict in your_results:
        for patient, data in data_dict.items():
            if patient in expected_results:
                correct_results += 1
            else:
                incorrect_results += 1

    print(f"Correct Results: {correct_results}")
    print(f"Incorrect Results: {incorrect_results}")
else:
    print("Failed to fetch expected results.")

Correct Results: 99
Incorrect Results: 0


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---